In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/4.part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1

                try:
                    resp = requests.get(f'''https://www.drivparts.com/driv/partfinder/api.catalog.product?brand=corporate&locale=en_US&country_code=US&part_number={input_.loc[a, 'Part Number']}&brand_code={input_.loc[a, 'Brand']}''',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))
                    
                    if resp.status_code == 200:
                        json_ = resp.json()

                        if json_:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            dict_specification, list_attribute = {}, json_['part_attributes'] if 'part_attributes' in json_ else []
            for i, attribute in zip(range(len(list_attribute)), list_attribute):
                dict_specification[str(i)] = {attribute['attribute'].strip(): f'''{attribute['attribute_value'].strip()} {attribute['attribute_uom_label'].strip()}''' if 'attribute_uom_label' in attribute else attribute['attribute_value'].strip()}
            
            # = = = = = = = = = = = = = = =

            dict_src, list_src = {}, json_['dam_assets']['productPrimaries']
            for i, src in zip(range(len(list_src)), list_src):
                dict_src[str(i)] = f'''https://www.drivparts.com{src['url'].strip()}'''

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'No': '',
                                     'Type': json_['part_type'].strip(),
                                     'Part Number': input_.loc[a, 'Part Number'],
                                     'Brand': input_.loc[a, 'Brand'],
                                     'Brand Name': json_['brand_name'].strip(),
                                     'GTIN': json_['gtin'].strip(),
                                     'Url': input_.loc[a, 'Url'],
                                     'Json_Src': json.dumps(dict_src)},
                                     'Json_Specification': json.dumps(dict_specification)])
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
            
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Type', 'Part Number'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct['No'] = [i+1 for i in range(len(output_correct))]
    output_correct.to_excel(f'''./file/5.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/part_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：60] - 7644.512305
[剩余数量：1000] - [当前时间：10:23:15]

[状态：ok，尝试次数：172] - 7618.512256
[剩余数量：999] - [当前时间：10:23:15]

[状态：ok，尝试次数：12] - 7677.512364
[剩余数量：998] - [当前时间：10:23:15]

[状态：ok，尝试次数：63] - 7670.512351
[剩余数量：997] - [当前时间：10:23:15]

[状态：ok，尝试次数：14] - 7691.512388
[剩余数量：996] - [当前时间：10:23:15]

[状态：ok，尝试次数：6] - 7693.512391
[剩余数量：995] - [当前时间：10:23:15]

[状态：ok，尝试次数：17] - 7681.512370
[剩余数量：994] - [当前时间：10:23:16]

[状态：ok，尝试次数：23] - 7686.512376
[剩余数量：993] - [当前时间：10:23:16]

[状态：ok，尝试次数：16] - 7696.512394
[剩余数量：992] - [当前时间：10:23:16]

[状态：ok，尝试次数：36] - 7689.512382
[剩余数量：991] - [当前时间：10:23:16]

[状态：ok，尝试次数：6] - 7700.512401
[剩余数量：990] - [当前时间：10:23:16]

[状态：ok，尝试次数：20] - 7697.512395
[剩余数量：989] - [当前时间：10:23:16]

[状态：ok，尝试次数：13] - 7707.512416
[剩余数量：988] - [当前时间：10:23:17]

[状态：ok，尝试次数：26] - 7698.512398
[剩余数量：987] - [当前时间：10:23:17]

[状态：ok，尝试次数：41] - 7645.512309
[剩余数量：986] - [当前时间：10:23:17]

[状态：ok，尝试次数：40] - 7690.512386
[剩余数量：985] - [当前时间：10:23:17]

[状态：ok，尝试次数：2] - 7714.512426
[剩余数量：984] 